# Attitude Sequence

This example is based on the attitude tutorial in the orekit library and webpage https://www.orekit.org/static/tutorial/attitude.html by Luc Maisonobe written in Java. This version of the example is based on ipython notebook, using a python-wrapped version of the orekit library, as a demonstration of the python capabilities and as a tutorial. Conversion made by Petrus Hyvönen, SSC, 2014.

This example uses AttitudesSequence that enables easy switching between attitude laws on event occurrences when propagating some SpacecraftState.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import math

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## Initialize orekit and the Java VM 

In [2]:
#initialize orekit and JVM
import orekit_jpype as orekit
orekit.initVM()

In [3]:
from orekit_jpype.pyhelpers import setup_orekit_data

In [4]:
setup_orekit_data()

In [5]:
from jpype import JImplements, JOverride

In [6]:
from org.hipparchus.geometry.euclidean.threed import RotationOrder
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.hipparchus.ode.events import Action

In [7]:
from org.orekit.attitudes import AttitudeProvider;
from org.orekit.attitudes import AttitudesSequence;
from org.orekit.attitudes import LofOffset;
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid;
from org.orekit.errors import OrekitException;
from org.orekit.frames import FramesFactory;
from org.orekit.frames import LOFType;
from org.orekit.orbits import KeplerianOrbit;
from org.orekit.orbits import Orbit;
from org.orekit.propagation import Propagator;
from org.orekit.propagation import SpacecraftState;
from org.orekit.propagation.analytical import EcksteinHechlerPropagator;
from org.orekit.propagation.events import EclipseDetector;
from org.orekit.propagation.events import EventDetector;
from org.orekit.propagation.events.handlers import EventHandler
from org.orekit.propagation.sampling import OrekitFixedStepHandler
from org.orekit.time import AbsoluteDate;
from org.orekit.time import TimeScalesFactory;
from org.orekit.utils import Constants, IERSConventions, AngularDerivativesFilter
from org.orekit.utils import PVCoordinates;
from org.orekit.utils import PVCoordinatesProvider

Let's set up an initial state as:

- a date in UTC time scale
- an orbit defined by the position and the velocity of the spacecraft in the EME2000 inertial frame and an associated central attraction coefficient chosen among many physical constants available in Orekit.

The initial orbit is here defined as a KeplerianOrbit.

In [8]:
initialDate =  AbsoluteDate(2004, 1, 1, 23, 30, 00.000, TimeScalesFactory.getUTC())
position  = Vector3D(-6142438.668, 3492467.560, -25767.25680);
velocity  = Vector3D(505.8479685, 942.7809215, 7435.922231);
initialOrbit =  KeplerianOrbit(PVCoordinates(position, velocity),
                                                          FramesFactory.getEME2000(), initialDate,
                                                          Constants.EIGEN5C_EARTH_MU);

In [9]:
print(initialOrbit)

Keplerian parameters: {a: 7069220.386682823; e: 4.777356060557311E-4; i: 98.18525099174988; pa: 13.741061002484528; raan: 150.34825333049; v: -13.952151446378437;}


More details on the orbit representation can be found in the orbits section of the library architecture documentation. 

Note that in the python wrapping of orekit the format of numbers is important, such that floats needs to be specified with a decimal point such as in the AbsoluteDate above where seconds are a float.

## Attitude Laws
Let's define a couple of AttitudeLaw for our satellite, one for daytime and one for nighttime, built upon Local Orbital Frame (LOF) Offset laws. VVLH is a coodrinate system based on Vehicle Velocity, Local Horizontal frame (Z axis aligned with nadir vector, X-Y is in horizontal plane, where the X-axis is along the velocity vector). 

The offset is defined in Cardan angles (RotationOrder.XYZ).

In [10]:
dayObservationLaw =  LofOffset(initialOrbit.getFrame(), 
                               LOFType.VVLH,
                               RotationOrder.XYZ, 
                               math.radians(20), math.radians(40), 0)

NameError: name 'math' is not defined

In night we want the pointing to be in LOF aligned. 

In [ ]:
nightRestingLaw   = LofOffset(initialOrbit.getFrame(), LOFType.VVLH)

## Event Detectors

Let's also define some EventDetectors for the switching. For this tutorial's requirements, two specialized event detectors are built 
upon an EclipseDetector:

one, dayNightEvent, to detect the day to night transition,
another, nightDayEvent, to detect the night to day transition.
This is done by overriding the eventOccurred method of the standard EclipseDetector. To override methods in a java class in Python,
a special wrapped version of the class is used, called PythonElevationDetector. When subclassing Orekit classes in Python, all methods of that interface needs to be defined, if so just with a "pass" statement.

In [ ]:
@JImplements(EventHandler)
class myNightEclipseDetector():

    @JOverride
    def init(self, s, T, detector):
        pass

    @JOverride
    def eventOccurred(self, s, detector, increasing):
        if not increasing:
            print("\n", s.getDate()," : event occurred, entering eclipse => switching to night law\n")
        return Action.CONTINUE
    
    @JOverride   
    def resetState(self, detector, oldState):
        return oldState;

In [ ]:
@JImplements(EventHandler)
class myDayEclipseDetector():
    @JOverride  
    def init(self, s, T, detector):
        pass
    
    @JOverride
    def eventOccurred(self, s, detector, increasing):
        if increasing:
            print("\n", s.getDate()," : event occurred, exiting eclipse => switching to day law\n")
        return Action.CONTINUE

    @JOverride
    def resetState(self, detector, oldState):
        return oldState

For the eclipse calculation we want to use the Sun and the Earth.

In [ ]:
sun = CelestialBodyFactory.getSun()

In [ ]:
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
                         0.0,
                         FramesFactory.getITRF(IERSConventions.IERS_2010, True))

Create the actual detectors with the the handlers specified above.

In [ ]:
dayNightEvent = EclipseDetector(sun, 696000000., earth)
dayNightEvent = dayNightEvent.withHandler(myNightEclipseDetector()) #.of_(EclipseDetector))

In [ ]:
nightDayEvent = EclipseDetector(sun, 696000000., earth)
nightDayEvent = nightDayEvent.withHandler(myDayEclipseDetector()) #.of_(EclipseDetector))

More details on event detectors can be found in the propagation section of the library architecture documentation.

## Attitude Sequence Definition

An AttitudesSequence is then defined, for the sake of this tutorial, by adding two switching conditions acting as a simple loop:

- the first one enables the transition from dayObservationLaw to nightRestingLaw when a decreasing dayNightEvent occurs,
- the second one enables the transition from nightRestingLaw to dayObservationLaw when an increasing nightDayEvent occurs.


In [ ]:
attitudesSequence = AttitudesSequence()

# Should add something more on Switching

attitudesSequence.addSwitchingCondition(dayObservationLaw, nightRestingLaw,
                                        dayNightEvent, False, True, 10.0, 
                                        AngularDerivativesFilter.USE_R, None)


attitudesSequence.addSwitchingCondition(nightRestingLaw, dayObservationLaw, nightDayEvent, True, False, 10.0, AngularDerivativesFilter.USE_R,  None)

An AttitudesSequence needs at least one switching condition to be meaningful, but there is no upper limit.

An active AttitudeLaw may have several switch events and next law settings, leading to different activation patterns depending on which event is triggered first.

So, don't forget to set the current active law according to the current state:

In [ ]:
if (dayNightEvent.g(SpacecraftState(initialOrbit)) >= 0):
    # initial position is in daytime
    attitudesSequence.resetActiveProvider(dayObservationLaw);
else:
    # initial position is in nighttime
    attitudesSequence.resetActiveProvider(nightRestingLaw);

## Orbit Propagation

Now, let's choose some propagator to compute the spacecraft motion. We will use an EcksteinHechlerPropagator based on the analytical Eckstein-Hechler model. The propagator is built upon the initialOrbit, the attitudeSequence and physical constants for the gravity field potential.

In [ ]:
propagator = EcksteinHechlerPropagator(initialOrbit, attitudesSequence,
                                                            Constants.EIGEN5C_EARTH_EQUATORIAL_RADIUS,
                                                            Constants.EIGEN5C_EARTH_MU, Constants.EIGEN5C_EARTH_C20,
                                                            Constants.EIGEN5C_EARTH_C30, Constants.EIGEN5C_EARTH_C40,
                                                            Constants.EIGEN5C_EARTH_C50, Constants.EIGEN5C_EARTH_C60)

The attitudeSequence must register all the switching events before propagation.

In [ ]:
# Register the switching events to the propagator
attitudesSequence.registerSwitchEvents(propagator)

The propagator operating mode is set to master mode with fixed step. The implementation of the interface OrekitFixedStepHandler aims to define the handleStep method called within the loop. For the purpose of this tutorial, the handleStep method will print at the current date two angles, the first one indicates if the spacecraft is eclipsed while the second informs about the current attitude law.

In [ ]:
@JImplements(OrekitFixedStepHandler)
class mystephandler():
    
    eclipseAngles = []
    pointingOffsets = []
    dates = []

    @JOverride
    def init(self,s0, t, step):
        pass

    @JOverride
    def handleStep(self,currentState):
        # the Earth position in spacecraft frame should be along spacecraft Z axis
        # during nigthtime and away from it during daytime due to roll and pitch offsets
        earth = currentState.toTransform().transformPosition(Vector3D.ZERO)
        pointingOffset = Vector3D.angle(earth, Vector3D.PLUS_K)

        # the g function is the eclipse indicator, its an angle between Sun and Earth limb,
        # positive when Sun is outside of Earth limb, negative when Sun is hidden by Earth limb
        eclipseAngle = dayNightEvent.g(currentState)
        print ("%s    %6.3f    %6.1f" % (currentState.getDate(), eclipseAngle, math.degrees(pointingOffset)))
        
        self.eclipseAngles.append(eclipseAngle)
        self.pointingOffsets.append(math.degrees(pointingOffset))
        self.dates.append(currentState.getDate())
        
    @JOverride
    def finish(self, s):
        pass

In [ ]:
handler = mystephandler()
propagator.setStepHandler(180.0, handler)

More details on propagation modes can be found in the propagation section of the library architecture documentation.

Finally, the propagator is just asked to propagate for a given duration.

In [ ]:
# Propagate from the initial date for the fixed duration
finalState = propagator.propagate(initialDate, initialDate.shiftedBy(12600.))

In [ ]:
print("Propagation ended at " + str(finalState.getDate().toString()))

In [ ]:
finalState

In [ ]:
ip = get_ipython()
ip.sphinxify_docstring = False
ip.enable_html_pager = False

In [ ]:
finalState

## Present Results

To plot nicely in Python, we should convert the orekit AbsoluteDate to python DateTime objects.

In [ ]:
from orekit_jpype.pyhelpers import absolutedate_to_datetime

In [ ]:
pydates = [absolutedate_to_datetime(t) for t in handler.dates]

In [ ]:
plt.xticks(rotation=90)
ax = plt.gca()
ax2 = ax.twinx()

ax.plot(pydates,handler.eclipseAngles, color='blue')
ax.set_ylabel('Eclipse Angle', color='blue')

ax2.plot(pydates, handler.pointingOffsets, color='green')
ax2.set_ylabel('Pointing offset', color='green');

The java code for similar example can be found in the source tree of the library, in file:

    src/tutorials/fr/cs/examples/attitude/EarthObservation.java
